In [ ]:
---
show-code: False
title: Zwizualizuj diagram BPMN
authors: Paweł Bucior, Julia Witek
description: Frontend do modelowania biznesowego
params:
    file_name:
        input: file
        label: Załaduj plik z danymi
---

# BPMN


In [1]:

!pip install opyenxes
!apt install libgraphviz-dev
# !apt install graphviz graphviz-dev
!pip install pygraphviz
from IPython.display import clear_output
clear_output(wait=False)

     |████████████████████████████████| 94 kB 2.8 MB/s 
  Created wheel for opyenxes: filename=opyenxes-0.3.0-py2.py3-none-any.whl size=79230 sha256=a06abb1cfcd19b064b5c4712e95c14018b845a2b96a7d3d7046670b652538447
  Stored in directory: /root/.cache/pip/wheels/b2/0b/80/cb6b1a80882eff1d3707d9b30ca5febe120d7fa0120e38e69b
Successfully built opyenxes
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common
  libgvc6-plugins-gtk libxdot4
Suggested packages:
  gvfs
The following NEW packages will be installed:
  libgail-common libgail18 libgraphviz-dev libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libgvc6-plugins-gtk libxdot4
0 upgraded, 8 newly installed, 0 to remove and 

In [2]:
file_name='example.csv'

In [ ]:
import pygraphviz as pgv
from itertools import product
from collections import defaultdict
from typing import Dict, Set
from IPython.display import Image, display

In [ ]:
from dateutil.parser import parse

def is_date(string, fuzzy=False):
    try:
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False


def process_csv(df):
    case_id = ''
    start_time = ''
    activity = ''

    if (case_id == '' or start_time == '' or activity == ''):
      for col in df.columns:
        if(len(df.columns)<3):
          print("Provided file cannot be used as an event log")

        if(len(df.columns)==3):
          if (df[col].dtypes == 'int64'):
            case_id = col
          elif (is_date(df.loc[0, col]) == True):
            start_time = col
          elif (df[col].dtypes != 'int64' and is_date(df.loc[0, col]) == False):
            activity = col
          else:
            case_id = input("Enter the name of id column: ")
            activity = input("Enter the name of activity column: ")
            start_time = input("Enter the name of timestamp column: ")

        if(len(df.columns)>3):
          if (df[col].dtypes == 'int64'):
            case_id = col
          elif (is_date(df.loc[0, col]) == True):
            start_time = col
            activity = input("Enter the name of activity column: ")
          else:
            case_id = input("Enter the name of id column: ")
            activity = input("Enter the name of activity column: ")
            start_time = input("Enter the name of timestamp column: ")

    print(start_time)
    print(case_id)
    print(activity)
    dfs = (df
    .sort_values(by=[case_id, start_time])
    .groupby([case_id])
    .agg({activity: ' '.join}))
    events = dfs.Activity.tolist()
    events_csv = []
    for line in events:
        events_csv.append(line.split())

    return events_csv


clear_output(wait=False)

In [ ]:
from opyenxes.data_in.XUniversalParser import XUniversalParser
import pandas as pd

f = open(file_name, "r")
if f.name.endswith('.csv'):
  print('csv')
  df = pd.read_csv(file_name)
  events = process_csv(df)
elif f.name.endswith('.xes'):
  print('xes')
  with open(file_name) as log_file:
    # parse the log
    log = XUniversalParser().parse(log_file)[0]
    event = log[0][0]
    event.get_attributes()
    events_xes = []
    for trace in log:
        events_w = []
        for event in trace[0::]:
            name = event.get_attributes()['Activity'].get_value()
            events_w.append(name)
        events_xes.append(events_w)
    events = events_xes
else:
  print('Nie rozpoznano pliku')


clear_output(wait=False)

In [ ]:
def get_start_set_events(events_tab):
  start_set_events = set()
  for line in events_tab:
    start_set_events.add(line[0])

  return start_set_events


def get_end_set_events(events_tab):
  end_set_events = set()
  for line in events_tab:
    end_set_events.add(line[len(line)-1])

  return end_set_events

def get_direct_successtion(events):
  direct_new = {}
  unique_elements = []
  for r in events:
   for c in r:
      if c not in unique_elements:
        unique_elements.append(c)
  unique_elements = sorted(unique_elements)
  for ue in unique_elements:
    relations = set()
    for r in events:
      for index, c in enumerate(r):
        if c == ue and int(len(r)) > int(index+1):
          relations.add(r[index+1])
    if len(relations) > 0:
      direct_new[ue] = relations
    else:
      direct_new[ue] = {}

  return direct_new


def get_causality(direct_succession) -> Dict[str, Set[str]]:
    causality = defaultdict(set)
    for ev_cause, events in direct_succession.items():
        for event in events:
            if ev_cause not in direct_succession.get(event, set()):
                causality[ev_cause].add(event)

    return dict(causality)


def get_potential_parallelism(direct_succession) -> Dict[str, Set[str]]:
  potential_parallelism = []
  for ev_cause, events in direct_succession.items():
    tmp = []
    for event in events:
      if ev_cause in direct_succession.get(event, set()):
        if ev_cause not in tmp:
          tmp.append(ev_cause)
        tmp.append(event)

    if len(tmp) > 0:
      potential_parallelism.append(tuple(tmp))
  if len(potential_parallelism) > 0:
    potential_parallelism.append(tuple(sorted(potential_parallelism[0])))

  return potential_parallelism


def get_inv_causality(causality) -> Dict[str, Set[str]]:
    inv_causality = defaultdict(set)
    for key, values in causality.items():
        for value in values:
          inv_causality[value].add(key)

    return {k: v for k, v in inv_causality.items() if len(v) > 1}


clear_output(wait=False)

In [ ]:
class MyGraph(pgv.AGraph):

    def __init__(self, *args):
        super(MyGraph, self).__init__(strict=False, directed=True, *args)
        self.graph_attr['rankdir'] = 'LR'
        self.node_attr['shape'] = 'Mrecord'
        self.graph_attr['splines'] = 'ortho'
        self.graph_attr['nodesep'] = '0.8'
        self.edge_attr.update(penwidth='2')

    def add_event(self, name):
        super(MyGraph, self).add_node(name, shape="circle", label="")

    def get_node(self,name):
        return super(MyGraph, self).get_node(name)

    def get_edges(self):
        return super(MyGraph, self).edges()

    def add_end_event(self, name):
        super(MyGraph, self).add_node(name, shape="circle", label="",penwidth='3')

    def add_and_gateway(self, *args):
        super(MyGraph, self).add_node(*args, shape="diamond",
                                  width=".7",height=".7",
                                  fixedsize="true",
                                  fontsize="40",label="+")

    def add_xor_gateway(self, *args, **kwargs):
        super(MyGraph, self).add_node(*args, shape="diamond",
                                  width=".7",height=".7",
                                  fixedsize="true",
                                  fontsize="40",label="×")
    def add_xor_inclusive_gateway(self, *args, **kwargs):
        super(MyGraph, self).add_node(*args, shape="diamond",
                                  width=".7",height=".7",
                                  fixedsize="true",
                                  fontsize="40",label="o")

    def add_event_gateway(self, source, targets, *args):
        gateway = 'EVENTs '+str(source)+'->'+str(targets)
        self.add_xor_inclusive_gateway(gateway,*args)
        super(MyGraph, self).add_edge(source, gateway)
        for target in targets:
            super(MyGraph, self).add_edge(gateway, target)

    def add_and_split_gateway(self, source, targets, *args):
        gateway = 'ANDs '+str(source)+'->'+str(targets)
        self.add_and_gateway(gateway,*args)
        super(MyGraph, self).add_edge(source, gateway)
        for target in targets:
            super(MyGraph, self).add_edge(gateway, target)

    def add_xor_split_gateway(self, source, targets, *args):
        gateway = 'XORs '+str(source)+'->'+str(targets)
        self.add_xor_gateway(gateway, *args)
        super(MyGraph, self).add_edge(source, gateway)
        for target in targets:
            super(MyGraph, self).add_edge(gateway, target)

    def add_event_merge_gateway(self, sources, target, *args):
        gateway = 'EVENTm '+str(sources)+'->'+str(target)
        self.add_xor_inclusive_gateway(gateway,*args)
        super(MyGraph, self).add_edge(gateway,target)
        for source in sources:
            super(MyGraph, self).add_edge(source, gateway)

    def add_and_merge_gateway(self, sources, target, *args):
        gateway = 'ANDm '+str(sources)+'->'+str(target)
        self.add_and_gateway(gateway,*args)
        super(MyGraph, self).add_edge(gateway,target)
        for source in sources:
            super(MyGraph, self).add_edge(source, gateway)

    def add_xor_merge_gateway(self, sources, target, *args):
        gateway = 'XORm '+str(sources)+'->'+str(target)
        self.add_xor_gateway(gateway, *args)
        super(MyGraph, self).add_edge(gateway,target)
        for source in sources:
            super(MyGraph, self).add_edge(source, gateway)

    def add_xor_to_and_gateway(self, sources, targets, *args):
        gateway_xor = 'XORm '+str(sources)+'->'+str(targets)
        gateway_and = 'ANDs '+str(sources)+'->'+str(targets)
        self.add_xor_gateway(gateway_xor, *args)
        self.add_and_gateway(gateway_and, *args)
        super(MyGraph, self).add_edge(gateway_xor,gateway_and)
        for source in sources:
            super(MyGraph, self).add_edge(source, gateway_xor)
        for target in targets:
            super(MyGraph, self).add_edge(gateway_and, target)


clear_output(wait=False)

In [ ]:
def draw_graph(events):

  # events_tab = []
  # for line in events.splitlines():
  #     events_tab.append(line.split())

  events_tab = events
  start_set_events = get_start_set_events(events_tab)
  end_set_events = get_end_set_events(events_tab)
  direct_succession = get_direct_successtion(events_tab)
  parallel_events = get_potential_parallelism(direct_succession)
  causality = get_causality(direct_succession)
  inv_causality = get_inv_causality(causality)

  G = MyGraph()

  # adding start event
  G.add_event("start")
  if len(start_set_events) > 1:
      if tuple(start_set_events) in sorted(parallel_events):
          G.add_and_split_gateway("start",start_set_events)

      else:
          G.add_xor_split_gateway("start",start_set_events)
  else:
      G.add_edge("start",list(start_set_events)[0])

  # adding split gateways based on causality
  for event in causality:
      if len(causality[event]) > 1 and tuple(causality[event]) != tuple(inv_causality.keys()):
          if tuple(causality[event]) in parallel_events:

              to_check = list(causality[event])
              is_event = False
              for eventeee in events_tab:
                if len(to_check) != len(list(set(sorted(to_check)).intersection(sorted(set(eventeee))))):
                  is_event = True

              if is_event:
                G.add_event_gateway(event,causality[event])
              else:
                G.add_and_split_gateway(event,causality[event])
          else:
              G.add_xor_split_gateway(event,causality[event])

  # adding merge gateways based on inverted causality
  for event in inv_causality:
      if len(inv_causality[event]) > 1 and tuple(sorted(inv_causality[event])) != tuple(sorted(causality.keys())):
          if tuple(inv_causality[event]) in parallel_events:
              to_check = list(inv_causality[event])
              is_event = False
              for eventeee in events_tab:
                if len(to_check) != len(list(set(sorted(to_check)).intersection(sorted(set(eventeee))))):
                  is_event = True

              if is_event:
                G.add_event_merge_gateway(inv_causality[event],event)
              else:
                G.add_and_merge_gateway(inv_causality[event],event)

          else:
              G.add_xor_merge_gateway(inv_causality[event],event)
      elif len(inv_causality[event]) == 1:
          source = list(inv_causality[event])[0]
          G.add_edge(source,event)

  xor_merge_gateways = set()
  for event in inv_causality:
    if (len(inv_causality[event]) > 1 and tuple(sorted(inv_causality[event])) == tuple(sorted(causality.keys()))):
      xor_merge_gateways.add(tuple(sorted(inv_causality[event])))

  # print(xor_merge_gateways)
  for create_gateway_event in xor_merge_gateways:
     G.add_xor_to_and_gateway(inv_causality[event],tuple(inv_causality.keys()))

  end_to_delete = False
  for end_event in end_set_events:
      for start_event in start_set_events:
        try:
          if tuple(causality[end_event]) == tuple(start_event):
            G.add_xor_split_gateway(end_event, (start_event,"end"))
            end_to_delete = True
        except:
          print('')

  # adding end event
  G.add_end_event("end")
  if len(end_set_events) > 1 and end_to_delete == False:
      if tuple(end_set_events) in parallel_events:
          G.add_and_merge_gateway(end_set_events,"end")
      else:
          G.add_xor_merge_gateway(end_set_events,"end")
  elif end_to_delete == False:
      G.add_edge(list(end_set_events)[0],"end")

  add_edges_list = []
  for sucession in direct_succession:
    found_count = 0
    for test in direct_succession:
      if test != sucession:
        if sucession in direct_succession[test]:
          found_count = found_count + 1
    if(found_count == len(direct_succession) -1):
      add_edges_list.append(sucession);

  for single_causality in causality:
    to_check = causality[single_causality]
    found_count = 1
    for test in causality:
      if test != single_causality:
        if to_check == causality[test]:
          found_count = found_count + 1
    if len(to_check) == 1 and found_count == 1 and list(to_check)[0] in add_edges_list:
      G.add_edge(single_causality,list(to_check)[0])

  G.draw('simple_process_model.png', prog='dot')
  display(Image('simple_process_model.png'))

clear_output(wait=False)

In [ ]:
draw_graph(events)